#建立项头表:
<p>第一次扫描数据,得到所有频繁1项集的计数.然后删除支持度低于阈值的项,将1项频繁集放入项头表,并按照支持度将许排列.第二次扫描数据,将读到的数据剔除非频繁1项集,并按照支持度降序排列.

In [4]:
import operator
import copy
Dataset=[({'面包'},{'牛奶'},{'茶'}), 
         ({'面包'},{'尿布'},{'啤酒'},{'茶'}),
         ({'牛奶'},{'尿布'},{'啤酒'}),
         ({'面包'},{'牛奶'},{'尿布'},{'茶'}),
         ({'面包'},{'尿布'},{'牛奶'}),
         ({'面包'},{'牛奶'},{'啤酒'},{'尿布'},{'茶'}),
         ({'啤酒'},{'牛奶'},{'茶'}),
         ({'面包'},{'茶'}),
         ({'面包'},{'尿布'},{'牛奶'},{'啤酒'},{'茶'}),
         ({'面包'},{'牛奶'})]
#统计支持度个数,并得到频繁1项集
frequent1_dict={} #商品字典：5种商品
non_frequent1_dict={}
goods_dict={}
for trade_set in Dataset: #统计每种商品的支持度
    for thing in trade_set:
        if tuple(thing) not in goods_dict.keys():
            goods_dict[tuple(thing)]=0
        goods_dict[tuple(thing)]+=1

for item in goods_dict.items():
    if (item[1]>0.35*len(Dataset)):
        frequent1_dict[item[0]]=item[1] #大于支持度的项集放入频繁列表
    else: 
        non_frequent1_dict[item[0]]=item[1]#把达不到minsup的项集放到非频繁项集里
print(frequent1_dict)
print(non_frequent1_dict)
freqItemSet=set(frequent1_dict)
print(freqItemSet)


{('面包',): 8, ('牛奶',): 8, ('茶',): 7, ('尿布',): 6, ('啤酒',): 5}
{}
{('茶',), ('尿布',), ('面包',), ('牛奶',), ('啤酒',)}


In [5]:
#FP树结点
class treeNode:
    def __init__(self, nameValue, numOccur, parentNode):
        self.name=nameValue     #存放节点名字,在构造时初始化为给定值
        self.count=numOccur      #用于计数出现次数,在构造时初始化为给定值
        self.nodeLink = None     #用于连接相似结点,指向下一个相似结点的指针
        self.parent = parentNode #指向父结点的指针
        self.children = {}       #存放孩子结点,指向子结点的字典,以子结点的元素名称为
        #为键,指向子结点的指针为值,初始化为空字典

    def inc(self, numOccur):#增加结点的出现次数
        self.count += numOccur

    def disp(self, ind=1):#仅用于输出调试
        print( ''*ind, self.name, ' ', self.count)
        for child in self.children.values():
            child.disp(ind+1)
            
    #更新头结点
    #nodeLink指向树中该元素项的每一个实例
    #从头指针表的nodeLink开始,一直沿着nodeLink直到到达链表末尾
    def updateHeader(self,nodeToTest,targetNode):
        while(nodeToTest.nodeLink!=None):#当测试结点后面还有结点时
            nodeToTest=nodeToTest.nodeLink#移动到下一个结点上
        nodeToTest.nodeLink=targetNode

    #更新FP树,把结点加入树里
    def updateFPtree(self,items,inTree,headerTable,count):
        #判断项集的第一个结点是否已作为根的子结点
        if item[0] in inTree.children:#如果是就加上去,计数增加1
            inTree.children[items[0]].inc(count)
        else: #如果不是根的孩子,就增加分枝
            inTree.children[items[0]]=treeNode(items[0],count,inTree)
            # 更新相应频繁项集的链表，往后添加
            if headerTable[items[0]][1] == None:#没有就新建
                headerTable[items[0]][1] = inTree.children[items[0]]
            else:
                updateHeader(headerTable[items[0]][1], inTree.children[items[0]])
        # 递归
        if len(items) > 1:
            updateFPtree(items[1::], inTree.children[items[0]], headerTable, count)

    def createTree(self,dataSet,minSup=1):
        "创建FP树"
        #第一次遍历数据集，创建头指针表
        headerTable={}
        for trade in dataSet:
            for item in trade:
                headerTable[item]=headerTable.get(item,0)+dataSet[trade]
        #移除不满足最小支持度的元素项
        for k in headerTable.keys():
            if headerTable[k]<minSup:
                del(headerTable[k])
        #增加一个数据项，用于存放指向相似元素项指针
        for key in headerTable:
            headerTable[key]=[headerTable[key],None]
        retTree=treeNode('Null Set',1,None)#根节点
        #第二次遍历数据集，创建FP树
        for tradeSet,count in dataSet.items():
            localD={}#对于一个项集tradeSet，记录其中每个元素项的全局频率，用于排序
            for item in tradeSet:
                if item in freqItemSet:
                    localD[item]=headerTable[item][0]#注意这个[0]，因为之前加过一个数据项
            if(len(localD)>0):
                orderedItems=[v[0] for v in sorted(localD.items(),key=lambda p:p[1],reverse=True)]
                updateTree(orderedItems,retTree,headerTable,count)#更新FP树
        return retTree,headerTable
    
    def loadSimpData(self):
        simpDat=[['面包', '牛奶', '茶'],
                 ['面包', '茶', '尿布', '啤酒'],
                 ['牛奶', '尿布', '啤酒'],
                 ['面包', '牛奶', '茶', '尿布'],
                 ['面包', '牛奶', '尿布'],
                 ['面包', '牛奶', '茶', '尿布', '啤酒'],
                 ['牛奶', '茶', '啤酒'],
                 ['面包', '茶'],
                 ['面包', '牛奶', '茶', '尿布', '啤酒'],
                 ['面包', '牛奶']]
        return simpDat

    def createInitSet(self,dataSet):
        retDict={}
        for trans in dataSet:
            retDict[frozenset(trans)] = 1
        return retDict
    
#实例化类
fpGrowth=treeNode('root',0,None)
fpGrowth.disp()
simpData=fpGrowth.loadSimpData()
print(simpData)
initSet=fpGrowth.createInitSet(simpData)
myFPtree,myHeaderTab=fpGrowth.createTree(initSet,1)
myFPtree.disp()

 root   0
[['面包', '牛奶', '茶'], ['面包', '茶', '尿布', '啤酒'], ['牛奶', '尿布', '啤酒'], ['面包', '牛奶', '茶', '尿布'], ['面包', '牛奶', '尿布'], ['面包', '牛奶', '茶', '尿布', '啤酒'], ['牛奶', '茶', '啤酒'], ['面包', '茶'], ['面包', '牛奶', '茶', '尿布', '啤酒'], ['面包', '牛奶']]
 Null Set   1


In [ ]:
#第二次扫描数据，对于每条数据剔除非频繁1项集，并按照支持度降序排列。
sortRule={'面包':0,'牛奶':1,'茶':2,'尿布':3,'啤酒':4}
DataSorted=[]
for affair in Dataset:#对于数据集中的每一条事务
    goods_list=[]#元组变成列表才能索引
    for non_freq_good in non_frequent1_dict:#删除非频繁1项集
        if non_freq_good in affair:
            affair.remove(non_freq_good)
    for good in affair:
        goods_list.append(list(good)[0])
    goods_list.sort(key=lambda x:sortRule[x])
    DataSorted.append(goods_list)
    print(goods_list)

构造FP树。从根节点∅开始，将过滤并排序后的样本一个个加入树中，若FP树不存在现有元素则添加分支，若存在则增加相应的值